In [1]:
import csv
import operator
import datetime
import os

from trainstop import *
from locations import *
from dateconversion import *

In [ ]:
#import data from all files
data_by_date = {}

for filename in os.listdir('./data'):
    date = datetime.date(int(filename[0:4]),int(filename[5:7]),int(filename[8:10]))
    file = open("data/" + date.isoformat() + "_IstDaten.csv")
    data_by_date[date] = csv.DictReader(file, delimiter=';')

#import data from files
train_stops = []
    
for date in data_by_date:
    for train in data_by_date[date]:
        train_stops.append(create_train_stop(train))


In [2]:
#import data from one file
file = open("data/2016-10-12_IstDaten.csv")
reader = csv.DictReader(file,delimiter=';')

train_stops = []
for train in reader:
    train_stops.append(TrainStop(train))

In [3]:
#count delay per stop

train_stops_delay = {}

for train_stop in train_stops:
    if train_stop not in train_stops_delay:
        train_stops_delay[train_stop.stop_name] = 0
    train_stops_delay[train_stop.stop_name] += train_stop.arrival_delay()
    
    

In [4]:
train_stops_delay

{'Aadorf': 2.0,
 'Aarau': 0.0,
 'Aarburg-Oftringen': 2.0,
 'Aathal': 0.0,
 'Aefligen': 6.0,
 'Aesch': 0.0,
 'Affoltern am Albis': 0.0,
 'Aigle': 0.0,
 'Airolo': 4.0,
 'Aix-en-Provence TGV': 0,
 'Aix-les-Bains-le-Revard': 0,
 'Albate-Camerlata': 0.0,
 'Allaman': -1.0,
 'Altdorf': 0.0,
 'Altendorf': 2.0,
 'Altstätten SG': -1.0,
 'Ambri-Piotta': -1.0,
 'Ambérieu': 0,
 'Amriswil': 1.0,
 'Amsterdam CS': 0.0,
 'Amstetten': 0,
 'Andelfingen': 1.0,
 'Antibes': 0,
 'Arnex': 2.0,
 'Arnhem': 1.0,
 'Arth-Goldau': 5.0,
 'Attnang-Puchheim': 0,
 'Au ZH': 0.0,
 'Auggen': 9.0,
 'Auvernier': 0.0,
 'Avenches': 0.0,
 'Avignon TGV': 0,
 'Avoudrey': 0,
 'Baar': 3.0,
 'Baar Lindenpark': 4.0,
 'Baar Neufeld': 3.0,
 'Bad Bellingen': 9.0,
 'Bad Krozingen': 7.0,
 'Bad Ragaz': 4.0,
 'Bad Zurzach': 0.0,
 'Baden': 0.0,
 'Baden-Baden': 1.0,
 'Baldegg': 1.0,
 'Baldegg Kloster': 1.0,
 'Balerna': 9.0,
 'Ballwil': 2.0,
 'Balsthal': -1.0,
 'Bartenheim': 0,
 'Basel Bad Bf': 0.0,
 'Basel Dreispitz': 2.0,
 'Basel SBB': 0.0,

In [5]:
sorted_by_delay = sorted(train_stops_delay.items(), key=operator.itemgetter(1))
sorted_by_delay

[('La Heutte', -3.0),
 ('Solothurn Allmend', -3.0),
 ('Neuchâtel', -2.0),
 ('Renens VD', -2.0),
 ('Marthalen', -2.0),
 ('Oberdorf SO', -2.0),
 ('Champ-du-Moulin', -2.0),
 ('Tenero', -2.0),
 ('Martigny', -2.0),
 ('Knonau', -2.0),
 ('Montreux', -2.0),
 ('Villeneuve', -2.0),
 ('Bern', -2.0),
 ('Winterthur Wallrüti', -2.0),
 ('Porrentruy', -2.0),
 ('Rheinfelden', -2.0),
 ('Olten', -1.0),
 ('Cheyres', -1.0),
 ('Rüti GL', -1.0),
 ('Altstätten SG', -1.0),
 ('Clarens', -1.0),
 ('Gordola', -1.0),
 ('Frick', -1.0),
 ('Brugg AG', -1.0),
 ('Crémines-Zoo', -1.0),
 ('Dompierre', -1.0),
 ('Les Hauts-Geneveys', -1.0),
 ('Schwanden GL', -1.0),
 ('Uznach', -1.0),
 ('Gänsbrunnen', -1.0),
 ('Romont', -1.0),
 ('Le Day', -1.0),
 ('St-Imier', -1.0),
 ('Grandval', -1.0),
 ('Le Pont', -1.0),
 ('Courfaivre', -1.0),
 ('Balsthal', -1.0),
 ('Lottstetten', -1.0),
 ('Signau', -1.0),
 ('Langnau', -1.0),
 ('La Plaine', -1.0),
 ('Villeret', -1.0),
 ('Hochdorf', -1.0),
 ('Le Brassus', -1.0),
 ('Rheinfelden Augarten', -1

In [6]:
train_stops_delay["Genève"]

5.0

In [7]:
station_names = []
for station_name in train_stops_delay:
    station_names.append(station_name)

save_to_file(get_locations(station_names))

We are 0.00% complete. Currect station : Au ZH
We are 0.00% complete. Currect station : Münchenstein
We are 0.00% complete. Currect station : Sélestat
We are 0.00% complete. Currect station : Münsingen
We are 0.00% complete. Currect station : Visp
We are 0.01% complete. Currect station : Neuhausen Rheinfall
We are 0.01% complete. Currect station : Neumünster
We are 0.01% complete. Currect station : Habsheim
We are 0.01% complete. Currect station : St-Saphorin
We are 0.01% complete. Currect station : Dole-Ville
We are 0.01% complete. Currect station : Boswil-Bünzen
We are 0.01% complete. Currect station : Frankfurt (M) Flughafen Fernb.
We are 0.01% complete. Currect station : Münster (Westf) Hbf
We are 0.01% complete. Currect station : Amriswil
We are 0.02% complete. Currect station : Verona Porta Nuova
We are 0.02% complete. Currect station : Olten
We are 0.02% complete. Currect station : Gisikon-Root
We are 0.02% complete. Currect station : Giubiasco
We are 0.02% complete. Currect sta